# Linear Optimization (CS5040) : Assignment 02

# Team:

#### 1. CS23MTECH11004 &emsp; - &emsp; Ashish B Emmanuel
#### 2. CS23MTECH11016 &emsp; - &emsp; Raghavendra Kulkarni
#### 3. CS23MTECH14013 &emsp; - &emsp; Shagun
#### 4. CS23MTECH14016 &emsp; - &emsp; Trishita Saha

# Question:

Implement the simplex algorithm to maximize the objective function under given constraints using Vertex Marching method and assuming:
* Polytope is non-degenerate
* Rank of A is n

# Input Format:

Input is a .csv file with m+2 rows and n+1 columns. 
* The first row excluding the last element is the initial feasible point z of length n
* The second row excluding the last element is the cost vector c of length n
* The last column excluding the top two elements is the constraint vector b of length m
* The rows third to m+2 and column one to n is the matrix A of size m*n

# Solution:

## Import the required libraries

In [1]:
import numpy as np
from sympy import Matrix, zoo

## Function to read the .csv Input File

In [2]:
'''
Name        :   readCSV()
Description :   Reads the input matrix A, vectors b and c and initial point z from a CSV file.
Argument    :   Name of the Input file (without path)
Return      :   The matrix A, vectors b and c and the initial point z in that order
'''

def readCSV(filename):
    
    # Open the file and assign a File Handler
    with open(filename) as input:

        # Read the first line to extract vector z
        line = input.readline()
        z = list(map(float, line.split(',')))
        z = np.asarray(z[:-1], dtype = float)

        # Read the second line to extract vector c
        line = input.readline()
        c = list(map(float, line.split(',')))
        c = np.asarray(c[:-1], dtype = float)

        # Read all the remaining lines to extract matrix A and vector b
        A, b = [], []
        for line in input.readlines():
            l = list(map(float, line.split(',')))
            A.append(l[:-1])
            b.append(l[-1])
        A = np.asarray(A, dtype = float)
        b = np.asarray(b, dtype = float)
    
    # Return the matrix and the vectors
    return A, b, c, z

## Function to test the feasibility of a given point

In [3]:
'''
Name        :   testFeasibility()
Description :   Checks whether a point z is a feasible solution for the system or not.
Argument    :   Matrix A, vector b and the point z to be checked
Return      :   True, if z is feasible
                False, otherwise
'''

def testFeasibility(A, b, z):
    
    # Multiply A with z and compare it with b. Return the truth value
    return all(np.matmul(A, z) <= b)

## Function to split the matrix into tight and untight rows

In [4]:
'''
Name        :   splitRows()
Description :   Splits the matrices A and b into set of tight and untight rows with respect to a given point z.
Argument    :   Matrix A, vector b and the point z
Return      :   Returns the set of tight and untight rows of A and b in that order
'''

def splitRows(A, b, z):
    
    # Initialize required variables
    A1, A2, b1, b2 = [], [], [], []

    # For every element in product Az
    for i in range(len(A)):

        # Compare with corresponding element of vector b and decide tight or untight
        if np.round(np.dot(A[i], z), 3) == np.round(b[i], 3):
            A1.append(A[i])
            b1.append(b[i])
        else:
            A2.append(A[i])
            b2.append(b[i])
    
    # Return the tight and untight rows 
    return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

## Function to test whether the given point is a vertex or not

In [5]:
'''
Name        :   testVertex()
Description :   Checks whether a point z is a vertex of the feasible region or not.
Argument    :   Matrix A, vector b and the point z to be checked
Return      :   True, if z is vertex
                False, otherwise
'''

def testVertex(A, b, z):

    # Split the matrix into tight and untight rows
    A1, _, _, _ = splitRows(A, b, z)

    # Compute the rank of matrix A and compare it with number of tight rows
    return np.linalg.matrix_rank(A) == A1.shape[0]

## Function to update a feasible point and move it towards a vertex

In [6]:
'''
Name        :   move()
Description :   Moves the feasible point towards a vertex of the feasible region by one step size.
Argument    :   Matrix A, vector b and the vertex z
Return      :   The point z reached after one step move
'''

def move(A, b, z):

    # Determine the shape of the matrix and split it into tight and untight rows
    _, n = A.shape[0], A.shape[1]
    A1, _, A2, b2 = splitRows(A, b, z)
    A1 = A1.reshape((A1.shape[0], n))

    # Compute the direction to move the feasible point
    nullspace = np.array(Matrix(A1).nullspace())
    scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
    vector = np.array(np.sum(nullspace * scale, axis = 0)).astype(float)
    u = vector / np.linalg.norm(vector)

    # Compute the step size 'alpha'
    temp = np.dot(A2 , u)
    temp = np.array([1e-16 if x == 0 else x for x in temp])
    alpha = (b2 -np.dot(A2, z))/temp.T
    alpha = alpha[0]
    alpha = np.array([a for a in alpha if not ((a > 10**6) or (-1*a) > 10**6)])
    alpha = alpha[np.argmin(np.abs(alpha.copy()))]

    # Update the feasible point by one step size
    u = u.T
    z = z + alpha * u
    z = z[0]

    # Return the updated feasible point
    return z

## Function to test whether the given vertex is an optimum vertex or not

In [7]:
'''
Name        :   testOptimum()
Description :   Checks whether a vertex z is an optimum vertex or not.
Argument    :   Matrix A, vector b and the vertex z to be checked
Return      :   True, if z is optimum vertex
                False, otherwise
'''

def testOptimum(A, b, z, c):
    
    # Split the matrix A into tight and untight rows
    A1, _, _, _ = splitRows(A, b, z)

    # Express the cost vector as a linear combination of tight rows
    beta = np.dot(np.linalg.inv(A1.T), c.T)

    # Check for non-negativity of all the coefficients and return the result
    return all([x >= 0 for x in beta])

## Function to march from one vertex to other optimizing the cost function

In [8]:
'''
Name        :   optimize()
Description :   Optimizes the cost function by moving to the next optimum vertex
Argument    :   Matrix A, vector b, the vertex z and the cost vector c
Return      :   The next optimum vertex
'''

def optimize(A, b, z, c):
    
    # Split the matrix into tight and untight rows
    A1, _, A2, b2 = splitRows(A, b, z)

    # Compute the direction to move in order to reach a more optimum vertex
    u = None
    A1_inverse = np.linalg.inv(A1)
    A1_inverse = np.transpose(A1_inverse)
    for x in A1_inverse:
        if np.dot((-1)*x, c.T) > 0:
            u = x.T
            break
    
    # Compute the step size 'alpha'
    temp = np.dot(A2, u)
    temp = [1e-16 if x == 0 else x for x in temp]
    alpha = (np.dot(A2, z) - b2)/temp
    alpha = [x for x in alpha if x != zoo]
    if all([a < 0 for a in alpha]):
        return z, False
    alpha = min([i for i in alpha if i >= 0])
    
    # Update the vertex
    z = z - alpha * u

    # Return the updated vertex
    return z, True

## The driver main function executing the Simplex Algorithm steps sequentially

In [9]:
'''
Name        :   main()
Description :   Driver function which calls the subroutines according to the Simplex Algorithm.
Argument    :   None
Return      :   None
'''
def main():

    ## Read the input from the CSV file
    A, b, c, z = readCSV('Assignment2.csv')
    
    # Step 1: Check for feasibility of the Initial Point
    if not testFeasibility(A, b, z):
        print('Initial point {} is not feasible...!!!'.format(z))
        return
    
    # Step 2: Move towards the first vertex
    while not testVertex(A, b, z):
        z = move(A, b, z)

    # Print the value of the Cost function at this first vertex
    print('Vertex {} --> Cost: {}'.format(z, np.dot(z, c)))

    # Step 3: March through every vertex and check for optimum
    solution = True
    while not testOptimum(A, b, z, c) and solution:
        z, solution = optimize(A, b, z, c)
        if not solution:
            print('No Solution')
        else:
            print('Vertex {} --> Cost: {}'.format(z, np.dot(z, c)))

    # Return
    return

## Invoking the driver main function

In [10]:
main()

Vertex [1. 0.] --> Cost: -3.0
